## Task One: Binary Representations

This task was all about working with bits. I had to implement four functions that manipulate bits in a 32-bit unsigned integer. These kinds of operations are essential in areas like cryptography and data compression, where you need to optimize data handling at the binary level.

### **Functions Implemented**
- **`rotl(x, n)`** – Rotates bits to the left.
- **`rotr(x, n)`** – Rotates bits to the right.
- **`ch(x, y, z)`** – Conditional bitwise selection.
- **`maj(x, y, z)`** – Computes the majority bit at each position.

Each function is essential in bitwise manipulations, particularly in hashing algorithms (e.g., SHA-256), data encoding, and optimizing storage.

### 1. **`rotl(x, n)`**
The rotate left (rotl) function shifts the bits of a number to the left by n positions. Bits that overflow on the left wrap around to the right. This is different from a left shift (<<), where overflowed bits are lost.

##### Why This Approach?
- Bitwise Shifting & Masking
Python supports [bitwise operations](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) for manipulating integers efficiently. Since Python integers are arbitrary precision, we use bitwise masking (& 0xFFFFFFFF) to limit the result to 32 bits, ensuring behavior similar to C-based implementations.

- Why is rotl useful in cryptographic functions?
Bitwise rotations are widely used in cryptographic hash functions like SHA-256 because they help spread input bits throughout the computation. This ensures diffusion, a property that makes it difficult for an attacker to reverse-engineer the input from the hash. Additionally, rotations preserve all bits of the input, unlike shifts (`<< or >>`), which discard bits.

- Handling Overflow Correctly
Since a [left shift](https://stackoverflow.com/questions/141525/what-are-bitwise-shift-bit-shift-operators-and-how-do-they-work) (`<<`) moves bits out of the range, I needed to reintroduce the overflowed bits using a right shift (`>>`) and a [bitwise OR](https://stackoverflow.com/questions/17484720/how-do-bitwise-or-and-bitwise-and-work-in-python) (`|`) to complete the rotation.



In [42]:

def rotl(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the left by n positions.

    Parameters:
        x (int): The 32-bit unsigned integer to rotate.
        n (int): The number of positions to rotate (default is 1).

    Returns:
        int: The rotated 32-bit unsigned integer.
    """
    n = n % 32  # Ensure n is within the valid range (0-31)
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF  # Perform bitwise rotation


- Example Usage Of Rotl
- This example rotates the bits of a number to the left by 4 positions and shows the result, including a case where all bits are set.


In [43]:
# Example usage of rotl
x = 0b00000000000000000000000000000001  # Binary representation of 1
rotated_value = rotl(x, 4)

# Output the original and rotated values
print(f"Original value (bin): {bin(x)}")
print(f"Rotated left by 4 (bin): {bin(rotated_value)}")

# Example with all bits set
x_all_set = 0xFFFFFFFF  # All 32 bits set to 1
rotated_all_set = rotl(x_all_set, 5)
print(f"Rotated left (all bits set): {bin(rotated_all_set)}") 


Original value (bin): 0b1
Rotated left by 4 (bin): 0b10000
Rotated left (all bits set): 0b11111111111111111111111111111111


#### Test Case For Rotate Left Function
##### The tests ensure the rotl function correctly rotates bits in a 32-bit unsigned integer:

- Basic Rotation: Checks if a simple left rotation shifts bits correctly.
- Wraparound Behavior: Verifies that overflowed bits wrap around to the right.
- Zero Rotation: Ensures rotating by 0 positions returns the original value.


In [44]:
import unittest 

class TestRotateLeft(unittest.TestCase):
    """
    Unit test class for testing the rotl (rotate left) function.
    """

    def test_rotl_basic(self):
        """
        Test case 1: Basic left rotation.
        - Input: `0b0001` (binary 1)
        - Rotating left by 4 positions
        - Expected Output: `0b10000` (binary 16)
        """
        self.assertEqual(rotl(0b0001, 4), 0b10000)

    def test_rotl_wraparound(self):
        """
        Test case 2: Check bit wraparound behavior.
        - Input: `0b10000000000000000000000000000000` (binary, MSB set)
        - Rotating left by 1 position
        - Expected Output: `0b00000000000000000000000000000001` (binary, LSB set)
        """
        self.assertEqual(rotl(0b10000000000000000000000000000000, 1), 
                         0b00000000000000000000000000000001)

    def test_rotl_zero_rotation(self):
        """
        Test case 3: Zero rotation (no changes).
        - Input: `0b1010` (binary 10)
        - Rotating left by 0 positions
        - Expected Output: `0b1010` (unchanged)
        """
        self.assertEqual(rotl(0b1010, 0), 0b1010)

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_ch_all_ones_selector (__main__.TestChooseFunction.test_ch_all_ones_selector)
Test when the selector is all 1s (should return y). ... ok
test_ch_all_zeros_selector (__main__.TestChooseFunction.test_ch_all_zeros_selector)
Test when the selector is all 0s (should return z). ... ok
test_ch_alternating_pattern (__main__.TestChooseFunction.test_ch_alternating_pattern)
Test with alternating selector bits. ... ok
test_ch_basic (__main__.TestChooseFunction.test_ch_basic)
Test basic choose function with a mix of 1s and 0s in the selector. ... ok
test_basic_words (__main__.TestHashFunction.test_basic_words)
Test common words to ensure correct hash values. ... ok
test_collisions (__main__.TestHashFunction.test_collisions)
Ensure that different inputs do not produce the same hash (minimize collisions). ... ok
test_empty_string (__main__.TestHashFunction.test_empty_string)
Test hashing an empty string should return 0. ... ok
test_long_string (__main__.TestHashFunction.test_long_string)
Test a l

### 2. **`rotr(x, n)`** 
The rotate right function shifts bits to the right by n positions. Bits that overflow on the right wrap around to the left.This ensures the integrity of data, unlike a right shift (>>), which discards shifted bits.

##### Why This Approach?
- Bitwise Shifting & Masking
Python supports [bitwise operations](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) for manipulating integers efficiently. Since Python integers are arbitrary precision, we use bitwise masking (& 0xFFFFFFFF) to limit the result to 32 bits, ensuring behavior similar to C-based implementations.

- Handling Overflow Correctly
Since a [right shift](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) (`>>`) moves bits out of range, I needed to reintroduce the overflowed bits using a [left shift](https://en.wikipedia.org/wiki/Circular_shift) (`<<`) and a [bitwise](https://en.wikipedia.org/wiki/Bitwise_operation#OR) OR (`|`) to complete the rotation and maintain proper circular shifting.


In [45]:
def rotr(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the right by n positions.

    Parameters:
        x (int): The 32-bit unsigned integer to rotate.
        n (int): The number of positions to rotate (default is 1).

    Returns:
        int: The rotated 32-bit unsigned integer.
    """
    n = n % 32  # Ensure n is within the valid range (0-31)
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF  # Perform bitwise rotation


##### Example Usage Of Rotr 
This example demonstrates rotating a number's bits to the right by 1 and 2 positions, including a case with a repeating bit pattern.


In [46]:
# Example usage of rotr
x = 0b10000000000000000000000000000000  # Binary representation with only the most significant bit set
rotated_value = rotr(x, 1)

# Output the original and rotated values
print(f"Original value (bin): {bin(x)}")
print(f"Rotated right by 1 (bin): {bin(rotated_value)}")

# Example with a pattern
x_pattern = 0b11001100110011001100110011001100
rotated_pattern = rotr(x_pattern, 2)
print(f"Original value (bin): {bin(x_pattern)}") 
print(f"Rotated right by 2 (bin): {bin(rotated_pattern)}")


Original value (bin): 0b10000000000000000000000000000000
Rotated right by 1 (bin): 0b1000000000000000000000000000000
Original value (bin): 0b11001100110011001100110011001100
Rotated right by 2 (bin): 0b110011001100110011001100110011


#### Test Case For Rotate Right Function
##### The tests ensure the rotr function correctly rotates bits in a 32-bit unsigned integer:

- Basic Rotation: Checks if a simple right rotation shifts bits correctly.
- Wraparound Behavior: Verifies that overflowed bits wrap around to the left.
- Zero Rotation: Ensures rotating by 0 positions returns the original value

In [47]:
import unittest

class TestRotateRight(unittest.TestCase):

    def test_rotr_basic(self):
        """Test basic right rotation by 1 position."""
        self.assertEqual(rotr(0b1000, 1), 0b0100)

    def test_rotr_wraparound(self):
        """Test wrap-around where the least significant bit moves to the most significant bit."""
        self.assertEqual(rotr(0b00000000000000000000000000000001, 1), 
                         0b10000000000000000000000000000000)

    def test_rotr_zero_rotation(self):
        """Test rotation by 0 positions (should return the original value)."""
        self.assertEqual(rotr(0b1010, 0), 0b1010)

unittest.main(argv=[''], verbosity=2, exit=False)


test_ch_all_ones_selector (__main__.TestChooseFunction.test_ch_all_ones_selector)
Test when the selector is all 1s (should return y). ... ok
test_ch_all_zeros_selector (__main__.TestChooseFunction.test_ch_all_zeros_selector)
Test when the selector is all 0s (should return z). ... ok
test_ch_alternating_pattern (__main__.TestChooseFunction.test_ch_alternating_pattern)
Test with alternating selector bits. ... ok
test_ch_basic (__main__.TestChooseFunction.test_ch_basic)
Test basic choose function with a mix of 1s and 0s in the selector. ... ok
test_basic_words (__main__.TestHashFunction.test_basic_words)
Test common words to ensure correct hash values. ... ok
test_collisions (__main__.TestHashFunction.test_collisions)
Ensure that different inputs do not produce the same hash (minimize collisions). ... ok
test_empty_string (__main__.TestHashFunction.test_empty_string)
Test hashing an empty string should return 0. ... ok
test_long_string (__main__.TestHashFunction.test_long_string)
Test a l


## 3. `ch(x, y, z)`  
The choose function (`ch`) is commonly used in cryptographic hashing (SHA-256). It selects bits from `y` or `z` based on `x`:  

- **If `x` has a `1`**, take the bit from `y`.  
- **If `x` has a `0`**, take the bit from `z`.  

### Why This Approach?  

- Cryptographic Reference
  The `ch` function follows the definition given in [FIPS PUB 180-4](https://csrc.nist.gov/publications/detail/fips/180/4/final), which specifies its use in secure hashing algorithms like SHA-256.  

- **Why is the choose function (ch) important for SHA-256?**
In SHA-256, `ch` helps introduce non-linearity by selecting bits from different registers based on the value of another register. This prevents the hash function from being predictable and helps maintain avalanche effect, where small input changes cause large output differences.

- Bitwise Selection Logic 
  In Python, bitwise operations allowed me to efficiently select values based on a mask. The function uses [bitwise AND/OR](https://docs.python.org/3/library/stdtypes.html#bitwise-operations) to implement this logic:  

  - `x & y`: Selects bits from `y` where `x` is `1`.  
  - `~x & z`: Selects bits from `z` where `x` is `0`.  
  - Combining them with `|` (**bitwise OR**) ensures the correct selection of bits.  



In [48]:
def ch(x: int, y: int, z: int) -> int:
    """
    Chooses bits from y where x has 1s, and from z where x has 0s.

    Parameters:
        x (int): Selector bits.
        y (int): Bits selected when x is 1.
        z (int): Bits selected when x is 0.

    Returns:
        int: The resulting integer after selection.
    """
    return (x & y) | (~x & z) & 0xFFFFFFFF  # Perform bitwise selection



#### Example Usage Of CH
This example demonstrates the ch function, which selects bits from y where x has 1s and from z where x has 0s, producing a result based on the selector bits in x.


In [49]:
# Example usage of ch
x = 0b10101010  # Selector bits
y = 0b11111111  # Bits to choose when x has 1s
z = 0b00000000  # Bits to choose when x has 0s

chosen_bits = ch(x, y, z)

# Output the result of the choose function
print(f"Selector (x):   {bin(x)}") 
print(f"Option 1 (y):   {bin(y)}")
print(f"Option 2 (z):   {bin(z)}")
print(f"Chosen result:  {bin(chosen_bits)}")

Selector (x):   0b10101010
Option 1 (y):   0b11111111
Option 2 (z):   0b0
Chosen result:  0b10101010


### Test Case for CH Function
These tests ensure that the ch function correctly selects bits from y or z based on the values in x, following the logic used in cryptographic hashing algorithms like SHA-256.

- Basic Selection: Verifies that bits from y are selected when x has 1s and bits from z are selected when x has 0s.
- All 1s in Selector (x): Ensures that when x is all 1s, the output is exactly y.
- All 0s in Selector (x): Ensures that when x is all 0s, the output is exactly z.
- Alternating Bit Pattern: Tests mixed bit patterns to check if the function properly selects bits from both y and z.

In [50]:
import unittest

class TestChooseFunction(unittest.TestCase):

    def test_ch_basic(self):
        """Test basic choose function with a mix of 1s and 0s in the selector."""
        self.assertEqual(ch(0b10101010, 0b11111111, 0b00000000), 0b10101010)

    def test_ch_all_ones_selector(self):
        """Test when the selector is all 1s (should return y)."""
        self.assertEqual(ch(0b11111111, 0b10101010, 0b01010101), 0b10101010)

    def test_ch_all_zeros_selector(self):
        """Test when the selector is all 0s (should return z)."""
        self.assertEqual(ch(0b00000000, 0b10101010, 0b01010101), 0b01010101)

    def test_ch_alternating_pattern(self):
        """Test with alternating selector bits."""
        self.assertEqual(ch(0b11001100, 0b11110000, 0b00001111), 0b11000011)

unittest.main(argv=[''], verbosity=2, exit=False)


test_ch_all_ones_selector (__main__.TestChooseFunction.test_ch_all_ones_selector)
Test when the selector is all 1s (should return y). ... ok
test_ch_all_zeros_selector (__main__.TestChooseFunction.test_ch_all_zeros_selector)
Test when the selector is all 0s (should return z). ... ok
test_ch_alternating_pattern (__main__.TestChooseFunction.test_ch_alternating_pattern)
Test with alternating selector bits. ... ok
test_ch_basic (__main__.TestChooseFunction.test_ch_basic)
Test basic choose function with a mix of 1s and 0s in the selector. ... ok
test_basic_words (__main__.TestHashFunction.test_basic_words)
Test common words to ensure correct hash values. ... ok
test_collisions (__main__.TestHashFunction.test_collisions)
Ensure that different inputs do not produce the same hash (minimize collisions). ... ok
test_empty_string (__main__.TestHashFunction.test_empty_string)
Test hashing an empty string should return 0. ... ok
test_long_string (__main__.TestHashFunction.test_long_string)
Test a l


### 4. **`maj(x, y, z)`** 
The majority function (`maj`) is widely used in cryptography and error correction:

- If at least two of x, y, z have a 1 at a bit position, the result is 1.
- Otherwise, the result is 0.

### **Why This Approach?**  

- Cryptographic Reference
The **majority (`maj`) function** is defined in [FIPS PUB 180-4]((https://csrc.nist.gov/publications/detail/fips/180/4/final)), which specifies its use in **SHA-256** and other secure hashing functions. It is a crucial component in cryptographic operations where bitwise majority logic helps strengthen message integrity.  

- Bitwise Logic in Python  
Python provides built-in [bitwise operations]((https://docs.python.org/3/library/stdtypes.html#bitwise-operations)) for efficient data manipulation. The `maj` function determines the majority bit at each position by computing:  
- `x & y`: Keeps bits set where **both `x` and `y` are 1**.  
- `x & z`: Keeps bits set where **both `x` and `z` are 1**.  
- `y & z`: Keeps bits set where **both `y` and `z` are 1**.  

By combining these with [`|` (bitwise OR)](https://docs.python.org/3/library/stdtypes.html#bitwise-or), the function ensures that **at least two of the three inputs contribute to a `1` at each bit position**, accurately computing the majority bitwise.  



In [51]:
def maj(x: int, y: int, z: int) -> int:
    """
    Computes the majority function at each bit position.

    Parameters:
        x (int): First input.
        y (int): Second input.
        z (int): Third input.

    Returns:
        int: The majority result.
    """
    return (x & y) | (x & z) | (y & z) & 0xFFFFFFFF  # Compute majority bitwise


### Example Usage Of MAJ
This example demonstrates the maj function, which outputs a 1 in each bit position where at least two of the inputs have a 1, showing how the majority vote works at the bit level.

In [52]:
# Example usage of maj
x = 0b10101010  # Input 1
y = 0b11110000  # Input 2
z = 0b00001111  # Input 3

majority_bits = maj(x, y, z)

# Output the result of the majority function
print(f"Input 1 (x):    {bin(x)}") 
print(f"Input 2 (y):    {bin(y)}")
print(f"Input 3 (z):    {bin(z)}")
print(f"Majority result: {bin(majority_bits)}")


Input 1 (x):    0b10101010
Input 2 (y):    0b11110000
Input 3 (z):    0b1111
Majority result: 0b10101010


### - Test Case For MAJ

The following test cases validate that the **`maj` function** correctly computes the majority bit for each position in a **32-bit unsigned integer**.

- Basic Majority Calculation: Ensures that when given a mix of `1s` and `0s`, the function correctly outputs the majority bits.
- All Zeros Input: Verifies that if **all inputs** are `0b00000000`, the function correctly returns `0b00000000`.
- All Ones Input: Ensures that when all inputs are **`1s`**, the function returns `0b11111111`.
- Two Majority Bits Agreement: Tests a scenario where **two inputs share majority bits while the third differs**, ensuring that the majority logic still holds.
- Mixed Bit Patterns: Checks that the function correctly computes majority bits when the inputs contain alternating patterns of `1s` and `0s`.  


In [53]:
import unittest

class TestMajorityFunction(unittest.TestCase):

    def test_maj_basic(self):
        """Test majority function with mixed input bits."""
        self.assertEqual(maj(0b10101010, 0b11110000, 0b00001111), 0b10101010)

    def test_maj_all_zeros(self):
        """Test when all inputs are zeros (result should be all zeros)."""
        self.assertEqual(maj(0b00000000, 0b00000000, 0b00000000), 0b00000000)

    def test_maj_all_ones(self):
        """Test when all inputs are ones (result should be all ones)."""
        self.assertEqual(maj(0b11111111, 0b11111111, 0b11111111), 0b11111111)

    def test_maj_two_majority_bits(self):
        """Test when two inputs agree on bits and the third differs."""
        self.assertEqual(maj(0b10101010, 0b10101010, 0b01010101), 0b10101010)

    def test_maj_pattern_mixed_bits(self):
        """Test a case with a mix of majority and minority bits."""
        self.assertEqual(maj(0b11001100, 0b11110000, 0b00001111), 0b11001100)

unittest.main(argv=[''], verbosity=2, exit=False)


test_ch_all_ones_selector (__main__.TestChooseFunction.test_ch_all_ones_selector)
Test when the selector is all 1s (should return y). ... ok
test_ch_all_zeros_selector (__main__.TestChooseFunction.test_ch_all_zeros_selector)
Test when the selector is all 0s (should return z). ... ok
test_ch_alternating_pattern (__main__.TestChooseFunction.test_ch_alternating_pattern)
Test with alternating selector bits. ... ok
test_ch_basic (__main__.TestChooseFunction.test_ch_basic)
Test basic choose function with a mix of 1s and 0s in the selector. ... ok
test_basic_words (__main__.TestHashFunction.test_basic_words)
Test common words to ensure correct hash values. ... ok
test_collisions (__main__.TestHashFunction.test_collisions)
Ensure that different inputs do not produce the same hash (minimize collisions). ... ok
test_empty_string (__main__.TestHashFunction.test_empty_string)
Test hashing an empty string should return 0. ... ok
test_long_string (__main__.TestHashFunction.test_long_string)
Test a l

---

## Task Two: Hash Functions

### Overview
##### This task explores hash functions, which are widely used in cryptography, data integrity, and efficient data lookup.

- Convert a given C hash function to Python.
- Test the Python implementation with different inputs.
- Explain why the numbers 31 and 101 were chosen in the function.

##### A hash function takes an input eg.a string and converts it into a fixed-size integer value. It should be:

- Efficient → Quickly compute a unique hash for an input.
- Deterministic → The same input always produces the same hash.
- Uniform → Hash values should be well distributed to minimize collisions.

### Understanding the C Function and Translation Process
##### The provided C function computes a hash value for a given string using a weighted sum approach and a modulo operation. The goal is to convert this function into Python while maintaining its logic and efficiency




```
The original C function:

unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}
```

### Translating to Python
Python does not use pointers, so i needed to adapt this function accordingly:

Key Adaptations:
- Use ord(char) to get the ASCII value (since char *s in C directly accesses ASCII).
- Iterate through the string using for char in s: instead of pointer arithmetic (*s != '\0').
- Keep integer calculations the same (31 * hashval + ASCII).
- Apply modulo 101 at the end to match the behavior.

In [54]:
def hash_function(s: str) -> int:
    """
    Implements the given C hash function in Python.

    Parameters:
        s (str): Input string.

    Returns:
        int: Hash value mod 101.
    """
    hashval = 0  # Initialize hash value
    for char in s:
        hashval = ord(char) + 31 * hashval  # Apply weighted sum with ASCII value
    return hashval % 101  # Use modulo to limit range


##### Test Cases for the Hash Function
To ensure the function works as expected, im going to test:

- Basic inputs → Common words like "hello", "world", and "python".
- Edge cases → Empty strings, single characters, and long inputs.
- Consistency → The same input should always produce the same hash.
- Collision checks → Different inputs should ideally result in different hash values.

In [55]:
import unittest

class TestHashFunction(unittest.TestCase):

    def test_basic_words(self):
        """Test common words to ensure correct hash values."""
        self.assertEqual(hash_function("hello"), 17)
        self.assertEqual(hash_function("world"), 34)
        self.assertEqual(hash_function("python"), 91)
        self.assertEqual(hash_function("hash"), 15)

    def test_empty_string(self):
        """Test hashing an empty string should return 0."""
        self.assertEqual(hash_function(""), 0)

    def test_single_character(self):
        """Test hashing single-character strings."""
        self.assertEqual(hash_function("a"), ord("a") % 101)
        self.assertEqual(hash_function("b"), ord("b") % 101)

    def test_long_string(self):
        """Test a long string to check consistency and stability."""
        self.assertEqual(hash_function("thisisalongstring"), hash_function("thisisalongstring"))

    def test_collisions(self):
        """Ensure that different inputs do not produce the same hash (minimize collisions)."""
        self.assertNotEqual(hash_function("apple"), hash_function("orange"))

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_ch_all_ones_selector (__main__.TestChooseFunction.test_ch_all_ones_selector)
Test when the selector is all 1s (should return y). ... ok
test_ch_all_zeros_selector (__main__.TestChooseFunction.test_ch_all_zeros_selector)
Test when the selector is all 0s (should return z). ... ok
test_ch_alternating_pattern (__main__.TestChooseFunction.test_ch_alternating_pattern)
Test with alternating selector bits. ... ok
test_ch_basic (__main__.TestChooseFunction.test_ch_basic)
Test basic choose function with a mix of 1s and 0s in the selector. ... ok
test_basic_words (__main__.TestHashFunction.test_basic_words)
Test common words to ensure correct hash values. ... ok
test_collisions (__main__.TestHashFunction.test_collisions)
Ensure that different inputs do not produce the same hash (minimize collisions). ... ok
test_empty_string (__main__.TestHashFunction.test_empty_string)
Test hashing an empty string should return 0. ... ok
test_long_string (__main__.TestHashFunction.test_long_string)
Test a l

### Why **31**?

The number **31** is used as a multiplier in the hash function, meaning each character’s ASCII value is multiplied by **31** before being added to the total hash value. The reason for this choice primarily lies in its **mathematical properties** and **computational efficiency**.

One of the most significant reasons for using **31** is that it is a **prime number**. Prime numbers are particularly useful in hashing because they help **reduce collisions**—situations where different inputs produce the same hash value. If a non-prime number were used, certain input patterns could create **predictable cycles**, leading to **clustering in the hash table**. By using a prime number, the **distribution of hash values is more uniform**, making the function more effective.  
📌 **Reference:** [Oracle Docs - Effective Java (Item 11: HashCode)](https://books.google.com/books?id=ka2VUBqHiWkC&pg=PA47&redir_esc=y#v=onepage&q&f=false) 

Another key reason is **computational efficiency**. In lower-level programming languages like **C**, multiplication by **31** can be optimized using **bitwise operations**, making it faster. Instead of performing direct multiplication, the expression:

```python
31 * hashval
```

can be rewritten using bitwise shifting:

```python
(hashval << 5) - hashval
```

This transformation works because shifting a value **left by 5 places** (`hashval << 5`) is equivalent to **multiplying it by 32**, and subtracting `hashval` effectively results in multiplying by **31**. This optimization allows compilers to replace multiplication with faster bitwise operations, improving performance.   

Although this optimization is **not necessary in Python**—since Python handles integer multiplication efficiently the use of **31** remains beneficial for maintaining consistency with C implementations and ensuring predictable results across different programming languages.   

Another reason **31** is useful is that it **spreads hash values more evenly** across the available range. When hashing a sequence of characters, using a small multiplier could lead to clustered hash values, meaning that similar inputs produce similar outputs. This lack of randomness increases the risk of hash collisions. Multiplying by **31** ensures that hash values are spread across a larger space, making collisions **less frequent**.  
📌 **Reference:** [Python Docs - Hashing](https://docs.python.org/3/glossary.html#term-hash-function)  





### Why 101?

The number **101** is used as the modulus in the hash function to ensure that the final hash values remain within a **bounded range** while maintaining an **even distribution**.

#### **Prime Number Properties**
101 is a **prime number**, which is crucial in hashing because it helps **reduce collisions**. Prime numbers minimize patterns in the distribution of hash values, making it less likely for different inputs to produce the same hash.

If a **non-prime number** were used, common divisors could lead to clustering, where hash values concentrate in specific areas of the range, reducing efficiency.

#### **Efficient Distribution & Uniformity**
Using 101 ensures that the hash values are evenly spread across **0 to 100**, which is ideal for creating hash tables with limited buckets. A poorly chosen modulo value could cause an*uneven spread, leading to performance bottlenecks.  
📌 **Reference:** [Princeton - Hashing Functions](https://algs4.cs.princeton.edu/34hash/)

#### **Avoiding Common Factors**
Numbers that share **common factors** with character sets or common word patterns may cause frequent collisions. Since **101 has no small divisors**, it avoids repeated patterns in the resulting hash values, ensuring a more random-like distribution.

#### **Modulo Keeps Hash Values Small**
Without the modulo operation (`% 101`), the hash value could **grow indefinitely**, making storage and computation inefficient. By keeping values within a fixed range, it improves lookup speed in hash tables and reduces memory usage.  
📌 **Reference:** [Python Docs - Modulo Operator](https://docs.python.org/3/library/stdtypes.html#numeric-types-int-float-complex)

---


## Task Three: SHA-256 Padding

### Overview

SHA-256 is a widely used cryptographic hash function that ensures data integrity and security. A crucial step in this hashing process is **message padding**, where the input data is adjusted to fit the SHA-256 block size of 512 bits before being processed.

The [SHA-256 padding process](https://csrc.nist.gov/publications/detail/fips/180/4/final) follows a specific format:

- Append a single `1` bit (`0x80`) to the message.  
- Pad with `0`s until the message length is 64 bits short of a multiple of 512.  
- Append the original message length as a 64-bit big-endian integer.  

This ensures that the input is correctly aligned for the SHA-256 compression function.

### Understanding the Padding Process
The padding process ensures that messages fit neatly into 512-bit blocks, which is critical for cryptographic security.

- Appending the 1 Bit
The first step is to mark the end of the original message by adding a **single `1` bit (`0x80`)**, ensuring that padding is unambiguous.

- Zero Padding to 448 Bits
The next step is to **add enough `0`s** so that the total length (including the appended `1` bit) is 448 bits mod 512. This aligns the message length correctly for SHA-256 processing.

- Appending the Length in Bits
The final step is to append the 64-bit representation of the original message length, ensuring that SHA-256 correctly interprets the data.

This method follows the exact structure described in the [NIST SHA-2 specification](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf), ensuring compliance with cryptographic standards.


- **Step 1: Read the File Contents**
    - I will start by opening the file and reading its contents as raw bytes

In [63]:
import os

def read_file(file_path: str):
    """
    Reads the contents of a file as bytes.

    Parameters:
        file_path (str): Path to the input file.

    Returns:
        bytes: File content in bytes.
    """
    with open(file_path, "rb") as file:
        return file.read()


- **Step 2: Compute the Original Bit Length**
    - SHA-256 requires the original message length in bits, so i will compute this from the byte length.

In [62]:
def compute_original_bit_length(data: bytes):
    """
    Computes the original message length in bits.

    Parameters:
        data (bytes): The input file content.

    Returns:
        int: Original length in bits.
    """
    return len(data) * 8


- **Step 3: Append the 1 Bit (0x80 in Hex)**
    - The SHA-256 specification states that a single 1 bit must be appended, which is represented as 0x80 in hex.

In [57]:
def append_one_bit(data: bytes):
    """
    Appends the 1-bit (0x80 in hex) to mark the end of the message.

    Parameters:
        data (bytes): The input file content.

    Returns:
        bytes: Data with the 1-bit appended.
    """
    return data + b'\x80'


- **Step 4: Add Zero Padding**
    - SHA-256 requires the total length (including the 1 bit) to be 448 mod 512. We compute how many zero bytes need to be added.

In [58]:
def add_zero_padding(padded_data: bytes):
    """
    Computes and appends the required zero padding.

    Parameters:
        padded_data (bytes): Data with the appended 1-bit.

    Returns:
        bytes: Data with zero padding.
    """
    zero_padding_length = (56 - (len(padded_data) % 64)) % 64
    return padded_data + (b'\x00' * zero_padding_length)


- **Step 5: Append the Original Length as a 64-bit Big-Endian Integer**
    - The final step is appending the original bit length, stored in big-endian format.

In [59]:
def append_original_length(padded_data: bytes, original_bit_length: int):
    """
    Appends the original length as a 64-bit big-endian integer.

    Parameters:
        padded_data (bytes): Data with zero padding.
        original_bit_length (int): Original message length in bits.

    Returns:
        bytes: Fully padded data.
    """
    return padded_data + original_bit_length.to_bytes(8, 'big')


- **Step 6: Extract the Padding**   
    - Now, I will extract and return only the padding bytes.

In [60]:
def extract_padding(original_data: bytes, padded_data: bytes):
    """
    Extracts only the padding portion from the fully padded message.

    Parameters:
        original_data (bytes): The original file content.
        padded_data (bytes): The fully padded message.

    Returns:
        str: The SHA-256 padding as a hex string.
    """
    padding_bytes = padded_data[len(original_data):]
    return " ".join(f"{byte:02X}" for byte in padding_bytes)


### Test Cases for SHA-256 Padding
##### To verify correctness, I will test:

- Basic input → Padding for "abc" as per SHA-256 specs.
- Edge cases → Empty strings, single characters, and near-block-size messages.
- Padding structure → Ensuring 0x80, zero padding, and length field are correctly placed.
- Consistency → Same input should always produce the same padding.
These tests confirm compliance with the SHA-256 specification.

In [64]:
# Testing SHA-256 Padding Functions
import unittest

class TestSHA256Padding(unittest.TestCase):

    def test_read_file(self):
        """Ensure the file is read correctly."""
        with open("test.txt", "wb") as f:
            f.write(b"abc")
        self.assertEqual(read_file("test.txt"), b"abc")

    def test_compute_original_bit_length(self):
        """Check if the function correctly calculates bit length."""
        self.assertEqual(compute_original_bit_length(b"abc"), 24)

    def test_append_one_bit(self):
        """Test appending the 1-bit to the message."""
        self.assertEqual(append_one_bit(b"abc"), b"abc\x80")

    def test_add_zero_padding(self):
        """Test zero padding length is correct."""
        padded_data = append_one_bit(b"abc")
        zero_padded_data = add_zero_padding(padded_data)
        self.assertEqual(len(zero_padded_data) % 64, 56)

    def test_append_original_length(self):
        """Ensure the correct 64-bit length is appended."""
        padded_data = append_one_bit(b"abc")
        padded_data = add_zero_padding(padded_data)
        final_data = append_original_length(padded_data, 24)
        self.assertEqual(final_data[-8:], (24).to_bytes(8, 'big'))

    def test_extract_padding(self):
        """Check if padding extraction works correctly."""
        original_data = b"abc"
        padded_data = append_one_bit(original_data)
        padded_data = add_zero_padding(padded_data)
        padded_data = append_original_length(padded_data, 24)
        padding_hex = extract_padding(original_data, padded_data)
        expected_padding = "80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18"
        self.assertEqual(padding_hex, expected_padding)

# Run tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_ch_all_ones_selector (__main__.TestChooseFunction.test_ch_all_ones_selector)
Test when the selector is all 1s (should return y). ... ok
test_ch_all_zeros_selector (__main__.TestChooseFunction.test_ch_all_zeros_selector)
Test when the selector is all 0s (should return z). ... ok
test_ch_alternating_pattern (__main__.TestChooseFunction.test_ch_alternating_pattern)
Test with alternating selector bits. ... ok
test_ch_basic (__main__.TestChooseFunction.test_ch_basic)
Test basic choose function with a mix of 1s and 0s in the selector. ... ok
test_basic_words (__main__.TestHashFunction.test_basic_words)
Test common words to ensure correct hash values. ... ok
test_collisions (__main__.TestHashFunction.test_collisions)
Ensure that different inputs do not produce the same hash (minimize collisions). ... ok
test_empty_string (__main__.TestHashFunction.test_empty_string)
Test hashing an empty string should return 0. ... ok
test_long_string (__main__.TestHashFunction.test_long_string)
Test a l

---

## Task 4: Prime Numbers
### Overview
Prime numbers play a crucial role in number theory and cryptography. Every integer greater than 1 can be uniquely expressed as a product of prime numbers, a property known as the **Fundamental Theorem of Arithmetic.**

In this task, I will compute the first 100 prime numbers using two well-known and efficient algorithms:

- Trial Division Algorithm – A basic but reliable method for checking primality.
- Sieve of Eratosthenes – A highly efficient algorithm for generating multiple prime numbers.

Both algorithms have different use cases. The **Trial Division Algorithm** is simple and useful for checking if a number is prime, whereas the **Sieve of Eratosthenes** is significantly faster when generating a large list of prime numbers. 

### Algorithm 1: Trial Division

#### How It Works  
The **Trial Division Algorithm** checks whether a given number \( n \) is prime by testing its divisibility from **2** up to **\( \sqrt{n} \)**. If no divisors are found within this range, the number is prime.

#### Steps:
1. If \( n \leq 1 \), return **False** (since 1 is not prime).
2. Check divisibility by **2**:
   - If \( n \) is **even** and greater than 2, return **False**.
3. Check **odd divisors** up to \( \sqrt{n} \):
   - If \( n \) is divisible by any of these numbers, return **False**.
4. If no divisors are found, return **True** (the number is prime).

#### Why This Approach?
- Simple and easy to understand.
- Works well for **small numbers** but becomes inefficient for **very large numbers**.
- Optimized by **checking only odd numbers** after 2, significantly reducing the number of divisions.


#### Trial Division: A straightforward method that checks divisibility up to √n.

In [2]:
import math

# Check if a number is prime using the Trial Division method
def is_prime_trial(n):
    """Returns True if 'n' is prime using the Trial Division method."""
    
    if n <= 1:
        return False
    if n == 2:  # 2 is the only even prime
        return True
    if n % 2 == 0:
        return False  # Exclude even numbers

    # Check divisibility for odd numbers up to √n
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False

    return True

#### Generate the first 'n' prime numbers using Trial Division

In [3]:
# Generate the first 'n' prime numbers using Trial Division
def first_n_primes_trial(n):
    """Generates the first 'n' prime numbers."""
    
    primes = []
    num = 2  # Start checking from 2

    while len(primes) < n:
        if is_prime_trial(num):
            primes.append(num)
        num += 1  # Move to the next number

    return primes


# Example: Find the first 100 prime numbers
primes_trial = first_n_primes_trial(100)
print(primes_trial)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
